In [101]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
from fuzzywuzzy import fuzz
from datetime import date
import datetime as dt
import pandas as pd
import numpy as np
import random
from selenium.webdriver.chrome.options import Options
import re
from tqdm import tqdm
import warnings
import selenium
warnings.filterwarnings("ignore")
from selenium.webdriver.support.ui import Select

In [102]:
driver = webdriver.Chrome()

In [103]:
url_seasons = ["https://www.sofascore.com/pt/torneio/futebol/england/premier-league/17#id:23776",
              "https://www.sofascore.com/pt/torneio/futebol/england/premier-league/17#id:29415",
              "https://www.sofascore.com/pt/torneio/futebol/england/premier-league/17#id:37036",
              "https://www.sofascore.com/pt/torneio/futebol/england/premier-league/17#id:41886",
              "https://www.sofascore.com/pt/torneio/futebol/england/premier-league/17#id:52186"]

In [104]:
url_seasons_dict = dict(zip(url_seasons,["2019-20","2020-21","2021-22","2022-23","2023-24"]))

In [111]:


def get_all_links(url):    
    all_links = pd.DataFrame(columns=["URL","SEASON_ID"])
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)
    
    button_elem = driver.find_elements("tag name","button")
    button_text = [val.text for val in button_elem]
    button_df = pd.DataFrame(zip(button_text,button_elem),columns=["TEXT","ELEMS"])


    links = pd.Series([val.get_attribute("href") for val in driver.find_elements("xpath","//a[@href]")])
    local_links = pd.DataFrame(links[links.str.contains("#id:")],columns=["URL"])
    local_links["SEASON_ID"] = url_seasons_dict[url]
    all_links = pd.concat([all_links, local_links])



    while len(button_df[button_df["TEXT"].str.contains("ANTER")]["ELEMS"].values)>0:


        links = pd.Series([val.get_attribute("href") for val in driver.find_elements("xpath","//a[@href]")])
        local_links = pd.DataFrame(links[links.str.contains("#id:")],columns=["URL"])        
        local_links["SEASON_ID"] = url_seasons_dict[url]
        
        all_links = pd.concat([all_links, local_links])

        button_elem = driver.find_elements("tag name","button")
        button_text = [val.text for val in button_elem]
        button_df = pd.DataFrame(zip(button_text,button_elem),columns=["TEXT","ELEMS"])  

        if len(button_df[button_df["TEXT"].str.contains("ANTER")]["ELEMS"].values)>0:
            button_df[button_df["TEXT"].str.contains("ANTER")]["ELEMS"].values[0].click()
            
    return all_links

In [112]:
url_seasons_0 = get_all_links(url_seasons[0])
url_seasons_1 = get_all_links(url_seasons[1])

url_seasons_2 = get_all_links(url_seasons[2])
url_seasons_3 = get_all_links(url_seasons[3])
url_seasons_4 = get_all_links(url_seasons[4])

In [124]:
driver = webdriver.Chrome()

In [ ]:
full_df_attack = pd.DataFrame()
full_df_defence = pd.DataFrame()
full_df_passing = pd.DataFrame()
full_df_duels = pd.DataFrame()
full_df_goalkeepers = pd.DataFrame()


count = 0

for url in tqdm(full_url["URL"].values[1800:]):

    driver.get(url)
    time.sleep(1)
    
    to_click = driver.find_elements("css selector","#__next > main > div.fresnel-container.fresnel-greaterThanOrEqual-mdMin > div.Box.cyOxcH.sc-d8bc48b6-0.dvhVZy > div.Box.Flex.ggRYVx.cQgcrM.sc-d8bc48b6-1.kYWPFe > div.Box.clAhaB.sc-d8bc48b6-2.cUcAWg > div:nth-child(2) > div > div.Box.haiiQU > div.Box.bkrWzf.sc-imWYAI.gzXbCV.secondary")
    
    if len(to_click)>0:
        
        to_click[0].click()

        time.sleep(1)

        button_elem = driver.find_elements("tag name","button")
        button_text = [val.text for val in button_elem]
        button_df = pd.DataFrame(zip(button_text,button_elem),columns=["TEXT","ELEMS"])  

        local_df_summary = pd.read_html(driver.page_source)[0]

        button_df[button_df["TEXT"].str.contains("Attack")]["ELEMS"].values[0].click()
        local_df_attack = pd.read_html(driver.page_source)[0]

        button_df[button_df["TEXT"].str.contains("Defen")]["ELEMS"].values[0].click()
        local_df_defence = pd.read_html(driver.page_source)[0]

        button_df[button_df["TEXT"].str.contains("Passin")]["ELEMS"].values[0].click()
        local_df_passing = pd.read_html(driver.page_source)[0]

        button_df[button_df["TEXT"].str.contains("Duel")]["ELEMS"].values[0].click()
        local_df_duels = pd.read_html(driver.page_source)[0]

        button_df[button_df["TEXT"].str.contains("Goalk")]["ELEMS"].values[0].click()
        local_df_goalkeepers = pd.read_html(driver.page_source)[0]

        matchup = driver.find_elements("tag name","h1")[0].text
        span_series = pd.Series([val.text for val in driver.find_elements("tag name","span")])
        round_matchup = span_series[span_series.str.contains("Round")].values[0]  

        local_df_attack["URL"] = url
        local_df_defence["URL"] = url
        local_df_passing["URL"] = url
        local_df_duels["URL"] = url
        local_df_goalkeepers["URL"] = url

        local_df_attack["MATCHUP"] = matchup
        local_df_defence["MATCHUP"] = matchup
        local_df_passing["MATCHUP"] = matchup
        local_df_duels["MATCHUP"] = matchup
        local_df_goalkeepers["MATCHUP"] = matchup

        local_df_attack["ROUND"] = round_matchup
        local_df_defence["ROUND"] = round_matchup
        local_df_passing["ROUND"] = round_matchup
        local_df_duels["ROUND"] = round_matchup
        local_df_goalkeepers["ROUND"] = round_matchup    

        full_df_attack = pd.concat([full_df_attack,local_df_attack])
        full_df_defence = pd.concat([full_df_defence,local_df_defence])
        full_df_passing = pd.concat([full_df_passing,local_df_passing])
        full_df_duels = pd.concat([full_df_duels,local_df_duels])
        full_df_goalkeepers = pd.concat([full_df_goalkeepers,local_df_goalkeepers])

        count+=1

        if count%100==1:
            full_df_attack.to_csv("full_attack.csv",index=False)
            full_df_defence.to_csv("full_defence.csv",index=False)
            full_df_passing.to_csv("full_passing.csv",index=False)
            full_df_duels.to_csv("full_duels.csv",index=False)
            full_df_goalkeepers.to_csv("full_goalkeepers.csv",index=False)      

In [ ]:
full_df_attack.to_csv("full_attack.csv",index=False)
full_df_defence.to_csv("full_defence.csv",index=False)
full_df_passing.to_csv("full_passing.csv",index=False)
full_df_duels.to_csv("full_duels.csv",index=False)
full_df_goalkeepers.to_csv("full_goalkeepers.csv",index=False)  